In [ ]:
!pip install evaluate

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

In [3]:
# Load processed data
df = pd.read_csv('/Users/karhin/Desktop/NYT_emotion/data/processed_dataset.csv')


In [5]:
df.head()

,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,emotion_5,emotion_6,emotion_7,emotion_8,headline_length,summary_length,clean_headline,clean_summary,clean_text,tokens,w2v_embedding,word_count,char_count,unique_word_ratio,sentiment_score
0,0,0,0,0,0,1,0,0,0,6,11,parenting lessons partial eclipse,don’t partiality maybe finally time try,parenting lessons partial eclipse don’t partia...,"['parenting', 'lessons', 'partial', 'eclipse',...",[ 0.2789099 0.35104534 0.5521695 0.729134...,10,73,1.000000,0.0000
1,0,0,0,0,0,1,0,0,0,2,8,catalonia,catalan lawmakers declared independence spain ...,catalonia catalan lawmakers declared independe...,"['catalonia', 'catalan', 'lawmakers', 'declare...",[-0.68591917 0.86837965 -0.2718815 -0.609799...,7,62,1.000000,0.0000
2,0,1,0,1,0,1,0,1,0,12,22,apple removes apps china store help internet u...,world’s valuable company appears pulled apps a...,apple removes apps china store help internet u...,"['apple', 'removes', 'apps', 'china', 'store',...",[-0.7135026 0.37232846 0.7488899 0.580454...,24,175,0.875000,0.7003
3,0,0,0,0,0,1,0,0,0,7,22,commerce secretary’s offshore ties putin ‘cron...,wilbur ross commerce secretary retained invest...,commerce secretary’s offshore ties putin ‘cron...,"['commerce', 'secretary', 's', 'offshore', 'ti...",[-8.28999802e-02 5.83586335e-01 1.88658923e-...,23,180,0.913043,0.0258
4,0,0,0,0,0,0,0,0,1,8,24,battered scarred ‘sphere’ returns 911 site,though sculpture emerged smoldering ruins larg...,battered scarred ‘sphere’ returns 911 site tho...,"['battered', 'scarred', 'sphere', 'returns', '...",[ 0.10605408 0.22822355 0.02063741 0.246064...,19,149,1.000000,-0.5859


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84194 entries, 0 to 84193
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   emotion_0          84194 non-null  int64  
 1   emotion_1          84194 non-null  int64  
 2   emotion_2          84194 non-null  int64  
 3   emotion_3          84194 non-null  int64  
 4   emotion_4          84194 non-null  int64  
 5   emotion_5          84194 non-null  int64  
 6   emotion_6          84194 non-null  int64  
 7   emotion_7          84194 non-null  int64  
 8   emotion_8          84194 non-null  int64  
 9   headline_length    84194 non-null  int64  
 10  summary_length     84194 non-null  int64  
 11  clean_headline     84194 non-null  object 
 12  clean_summary      84194 non-null  object 
 13  clean_text         84194 non-null  object 
 14  tokens             84194 non-null  object 
 15  w2v_embedding      84194 non-null  object 
 16  word_count         841

In [7]:
# Define feature and target columns
feature_columns = ['headline_length', 'summary_length', 'word_count', 'char_count', 'unique_word_ratio', 'sentiment_score']
target_columns = ['emotion_0', 'emotion_1', 'emotion_2', 'emotion_3', 'emotion_4', 'emotion_5', 'emotion_6', 'emotion_7', 'emotion_8']

X = df[feature_columns]
y = df[target_columns]


In [9]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
print(train_dataset[0])  # Check that 'labels' are float tensors


{'clean_text': 'congo’s specialty brews look ‘future coffee’ despite conflict political instability democratic republic congo believed one africa’s promising specialty coffee producers', 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'input_ids': [101, 9030, 1521, 1055, 12233, 24702, 2015, 2298, 1520, 2925, 4157, 1521, 2750, 4736, 2576, 18549, 3537, 3072, 9030, 3373, 2028, 3088, 1521, 1055, 10015, 12233, 4157, 6443, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [71]:
# Convert target columns to float32 for BCEWithLogitsLoss
df[target_columns] = df[target_columns].astype(np.float32)

# Add labels column as a list of emotions
df['labels'] = df[target_columns].values.tolist()


In [75]:
def tokenize_function(examples):
    tokenized_text = tokenizer(examples["clean_text"], padding="max_length", truncation=True)
    tokenized_text["labels"] = examples["labels"]
    return tokenized_text

dataset = Dataset.from_pandas(df[['clean_text', 'labels']])
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['clean_text'])


Map:   0%|          | 0/84194 [00:00<?, ? examples/s]

In [77]:
import torch

def format_dataset(example):
    example['labels'] = torch.tensor(example['labels'], dtype=torch.float32)
    return example

tokenized_dataset = tokenized_dataset.map(format_dataset)


Map:   0%|          | 0/84194 [00:00<?, ? examples/s]

In [79]:
tokenized_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 84194
})

In [81]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]


In [83]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=len(target_columns), 
    problem_type="multi_label_classification"
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
print(train_dataset[0])  # Ensure 'labels' are float tensors


{'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'input_ids': [101, 4372, 14535, 9295, 16067, 4641, 6613, 17710, 11031, 6047, 4386, 15962, 12968, 5094, 9295, 16067, 3201, 2034, 4935, 6613, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [87]:
# Convert target columns to float format for BCEWithLogitsLoss
df[target_columns] = df[target_columns].astype(np.float32)

# Prepare dataset for Hugging Face
dataset = Dataset.from_pandas(df[['clean_text'] + target_columns])


In [89]:
def tokenize_function(examples):
    tokenized_text = tokenizer(examples["clean_text"], padding="max_length", truncation=True)
    # Convert labels to float32
    labels = {key: float(value) for key, value in examples.items() if key.startswith('emotion_')}
    tokenized_text["labels"] = list(labels.values())
    return tokenized_text


In [91]:
import torch

def format_dataset(example):
    example['labels'] = torch.tensor(example['labels'], dtype=torch.float32)
    return example

tokenized_dataset = tokenized_dataset.map(format_dataset)


Map:   0%|          | 0/84194 [00:00<?, ? examples/s]

In [93]:
print(train_dataset[0])  # Check that 'labels' are float tensors


{'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'input_ids': [101, 4372, 14535, 9295, 16067, 4641, 6613, 17710, 11031, 6047, 4386, 15962, 12968, 5094, 9295, 16067, 3201, 2034, 4935, 6613, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [95]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()
results = trainer.evaluate()
print("Evaluation Results:", results)


Epoch,Training Loss,Validation Loss
1,0.361600,0.327503
2,0.324100,0.325597
3,0.341200,0.329031


Evaluation Results: {'eval_loss': 0.32903143763542175, 'eval_runtime': 413.7091, 'eval_samples_per_second': 40.703, 'eval_steps_per_second': 5.088, 'epoch': 3.0}
